<a href="https://colab.research.google.com/github/nasibehmohammadi/Thesis/blob/main/1GRU_d1(kaggle).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

file_path = "/content/d1.csv"
df = pd.read_csv(file_path)

print("🔍 بررسی ۵ سطر اول دیتاست:")
print(df.head())

feature_columns = ["B","C","D","E","F","G","H","I","J"]
target_column = "A"

X_data = df[feature_columns].values
y_data = df[target_column].values

num_features = X_data.shape[1]

time_steps = 10

X_sequences = []
y_sequences = []

for i in range(len(X_data) - time_steps):
    X_sequences.append(X_data[i : i + time_steps])
    y_sequences.append(y_data[i + time_steps])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = X_train.reshape(-1, num_features)
X_test = X_test.reshape(-1, num_features)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(-1, time_steps, num_features)
X_test = X_test.reshape(-1, time_steps, num_features)

model = Sequential([
    GRU(64, return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    GRU(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ دقت مدل روی داده‌های تست: {accuracy * 100:.2f}%")

test_sample = X_test[0].reshape(1, time_steps, num_features)
prediction = model.predict(test_sample)

if prediction > 0.5:
    print("🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.")
else:
    print("✅ ورود ایمن است.")


🔍 بررسی ۵ سطر اول دیتاست:
   A      B      C       D       E       F       G       H       I       J
0  1  39353  85475  117961  118300  123472  117905  117906  290919  117908
1  1  17183   1540  117961  118343  123125  118536  118536  308574  118539
2  1  36724  14457  118219  118220  117884  117879  267952   19721  117880
3  1  36135   5396  117961  118343  119993  118321  240983  290919  118322
4  1  42680   5905  117929  117930  119569  119323  123932   19793  119325


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.9381 - loss: 0.2483 - val_accuracy: 0.9435 - val_loss: 0.2204
Epoch 2/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.9425 - loss: 0.2235 - val_accuracy: 0.9435 - val_loss: 0.2178
Epoch 3/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.9430 - loss: 0.2208 - val_accuracy: 0.9435 - val_loss: 0.2177
Epoch 4/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.9417 - loss: 0.2240 - val_accuracy: 0.9435 - val_loss: 0.2176
Epoch 5/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.9396 - loss: 0.2304 - val_accuracy: 0.9435 - val_loss: 0.2174
Epoch 6/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.9429 - loss: 0.2208 - val_accuracy: 0.9435 - val_loss: 0.2171
Epoch 7/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9398 - loss: 0.2287 - val_accuracy: 0.9435 - val_loss: 0.2172
Epoch 8/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.9401 - loss: 0.2288 - 

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from sklearn.metrics import f1_score

# 1️⃣ پیش‌بینی بر روی مجموعه تست
y_pred = model.predict(X_test)

# 2️⃣ تبدیل احتمالات به برچسب‌های 0 و 1 (اگر مقدار احتمال > 0.5 باشد، کلاس 1 است، در غیر این صورت 0)
y_pred_classes = (y_pred > 0.5).astype(int)

# 3️⃣ محاسبه F1-Score
f1 = f1_score(y_test, y_pred_classes)

print(f"🎯 F1-Score مدل: {f1:.4f}")

205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
🎯 F1-Score مدل: 0.9709
